# 4-Channel MR Latent Dataset Creation

This notebook creates a dataset using all 4 channels from the MR latent representation for richer MR→CT synthesis.

In [1]:
import numpy as np
import SimpleITK as sitk
import os
import pickle
from pathlib import Path

print("📦 Libraries imported successfully")
print(f"📍 Current directory: {os.getcwd()}")

📦 Libraries imported successfully
📍 Current directory: /Users/samwallace/Documents/sCT/Comparison Pix2Pix/3D/Inspection


## 📊 Load and Inspect Original Data

In [4]:
# Paths to your original data
mr_latent_path = "/Users/samwallace/Documents/sCT/Comparison Pix2Pix/3D/datasets/overfit/1ABA009_latent_mr.nii"
ct_path = "/Users/samwallace/Documents/sCT/Comparison Pix2Pix/3D/datasets/overfit/CT_normalised_latent.mha"

print("🔍 Checking file existence:")
print(f"MR latent file: {os.path.exists(mr_latent_path)} - {mr_latent_path}")
print(f"CT file: {os.path.exists(ct_path)} - {ct_path}")

if not os.path.exists(mr_latent_path):
    print("❌ MR latent file not found. Please check the path.")
if not os.path.exists(ct_path):
    print("❌ CT file not found. Please check the path.")

🔍 Checking file existence:
MR latent file: True - /Users/samwallace/Documents/sCT/Comparison Pix2Pix/3D/datasets/overfit/1ABA009_latent_mr.nii
CT file: True - /Users/samwallace/Documents/sCT/Comparison Pix2Pix/3D/datasets/overfit/CT_normalised_latent.mha


In [5]:
# Load the MR latent data (4D with 4 channels)
mr_sitk = sitk.ReadImage(mr_latent_path)
mr_array = sitk.GetArrayFromImage(mr_sitk)

# Load the CT data 
ct_sitk = sitk.ReadImage(ct_path)
ct_array = sitk.GetArrayFromImage(ct_sitk)

print("📊 Data shapes:")
print(f"MR latent shape: {mr_array.shape}")
print(f"CT shape: {ct_array.shape}")

# Check if MR has channels
if len(mr_array.shape) == 4:
    print(f"✅ MR data has {mr_array.shape[-1]} channels - Perfect for 4-channel processing!")
    print(f"   Spatial dimensions: {mr_array.shape[:-1]}")
elif len(mr_array.shape) == 3:
    print(f"⚠️ MR data is 3D - might need to add channel dimension")

print(f"📏 MR data range: [{np.min(mr_array):.3f}, {np.max(mr_array):.3f}]")
print(f"📏 CT data range: [{np.min(ct_array):.3f}, {np.max(ct_array):.3f}]")

📊 Data shapes:
MR latent shape: (32, 128, 128, 4)
CT shape: (32, 128, 128)
✅ MR data has 4 channels - Perfect for 4-channel processing!
   Spatial dimensions: (32, 128, 128)
📏 MR data range: [-4.169, 4.847]
📏 CT data range: [-1.000, 1.000]


## 🧩 Create 4-Channel Dataset

In [7]:
# Create output directory
output_dir = "../datasets/all_channels"

os.makedirs(output_dir, exist_ok=True)

print(f"📁 Created output directory: {output_dir}")

# Define crop parameters (you can adjust these)
# Using the full available data
if len(mr_array.shape) == 4:
    z_size, y_size, x_size, channels = mr_array.shape
    crop_bounds = [0, z_size, 0, y_size, 0, x_size]
else:
    z_size, y_size, x_size = mr_array.shape
    crop_bounds = [0, z_size, 0, y_size, 0, x_size]
    channels = 1

print(f"📐 Data dimensions: {z_size}×{y_size}×{x_size} with {channels} channels")
print(f"📋 Crop bounds: {crop_bounds}")

📁 Created output directory: ../datasets/all_channels
📐 Data dimensions: 32×128×128 with 4 channels
📋 Crop bounds: [0, 32, 0, 128, 0, 128]


In [8]:
# Save 4-channel MR data
mr_filename = "1HNA001_mr_all_channels.npz"
mr_output_path = os.path.join(output_dir, mr_filename)

# Ensure we have 4D data (D, H, W, C)
if len(mr_array.shape) == 4:
    # Already has channels
    mr_data_to_save = mr_array
    print(f"💾 Saving 4-channel MR data: {mr_array.shape}")
elif len(mr_array.shape) == 3:
    # Add channel dimension - but this means we don't have 4 channels
    mr_data_to_save = mr_array[..., np.newaxis]
    print(f"⚠️ Only 3D data available, adding single channel: {mr_data_to_save.shape}")

np.savez_compressed(mr_output_path, data=mr_data_to_save)
print(f"✅ Saved MR data to: {mr_output_path}")

# Save CT data
ct_filename = "1HNA001_ct.npz"
ct_output_path = os.path.join(output_dir, ct_filename)

np.savez_compressed(ct_output_path, data=ct_array)
print(f"✅ Saved CT data to: {ct_output_path}")

# Verify saved data
mr_loaded = np.load(mr_output_path)['data']
ct_loaded = np.load(ct_output_path)['data']

print(f"\n🔍 Verification:")
print(f"MR saved shape: {mr_loaded.shape}")
print(f"CT saved shape: {ct_loaded.shape}")
print(f"MR channels available: {mr_loaded.shape[-1] if len(mr_loaded.shape) == 4 else 1}")

💾 Saving 4-channel MR data: (32, 128, 128, 4)
✅ Saved MR data to: ../datasets/all_channels/1HNA001_mr_all_channels.npz
✅ Saved CT data to: ../datasets/all_channels/1HNA001_ct.npz

🔍 Verification:
MR saved shape: (32, 128, 128, 4)
CT saved shape: (32, 128, 128)
MR channels available: 4


## 📋 Create Dataset Metadata

In [9]:
# Create the crops.pkl file for the dataset
sample_data = {
    'name': '1HNA001',
    'mr_path': mr_filename,
    'ct_path': ct_filename,
    'bounds': crop_bounds,
    'channels': 'all_4_channels',
    'channel_count': mr_loaded.shape[-1] if len(mr_loaded.shape) == 4 else 1
}

samples_list = [sample_data]

# Save the metadata
crops_pkl_path = os.path.join(output_dir, "crops.pkl")
with open(crops_pkl_path, 'wb') as f:
    pickle.dump(samples_list, f)

print(f"✅ Saved dataset metadata to: {crops_pkl_path}")
print(f"📊 Sample data: {sample_data}")

# Verify the pickle file
with open(crops_pkl_path, 'rb') as f:
    loaded_samples = pickle.load(f)
    
print(f"\n🔍 Verification - loaded {len(loaded_samples)} samples:")
for i, sample in enumerate(loaded_samples):
    print(f"Sample {i}: {sample}")

✅ Saved dataset metadata to: ../datasets/all_channels/crops.pkl
📊 Sample data: {'name': '1HNA001', 'mr_path': '1HNA001_mr_all_channels.npz', 'ct_path': '1HNA001_ct.npz', 'bounds': [0, 32, 0, 128, 0, 128], 'channels': 'all_4_channels', 'channel_count': 4}

🔍 Verification - loaded 1 samples:
Sample 0: {'name': '1HNA001', 'mr_path': '1HNA001_mr_all_channels.npz', 'ct_path': '1HNA001_ct.npz', 'bounds': [0, 32, 0, 128, 0, 128], 'channels': 'all_4_channels', 'channel_count': 4}


## 🚀 Ready for 4-Channel Training!

Your 4-channel dataset is now ready for training with the new `networks_3d_4channel.py`!

In [ ]:
# Generate training command for 4-channel model
print("🎯 Training Commands for 4-Channel Model:")
print("=" * 60)

# Basic training command
basic_cmd = f"""
# Basic 4-channel training:
python ../launch_4channel_training.py \\
    --name 4channel_mr_to_ct \\
    --dataroot {output_dir} \\
    --architecture unet_4channel_128 \\
    --batch_size 1 \\
    --lr 0.0002 \\
    --niter 50 \\
    --niter_decay 50 \\
    --gpu_ids 0
"""

# Advanced training command
advanced_cmd = f"""
# Advanced 4-channel training with augmentation:
python ../launch_4channel_training.py \\
    --name 4channel_mr_to_ct_advanced \\
    --dataroot {output_dir} \\
    --architecture resnet_4channel_6blocks \\
    --batch_size 2 \\
    --lr 0.0001 \\
    --niter 100 \\
    --niter_decay 100 \\
    --augment \\
    --normalize \\
    --mixed_precision \\
    --gpu_ids 0
"""

print(basic_cmd)
print("\n" + "=" * 60)
print(advanced_cmd)

print("\n📁 Files Created for 4-Channel Training:")
files_created = [
    "../models/networks_3d_4channel.py",
    "../train_4channel.py", 
    "../data/four_channel_dataset.py",
    "../options/four_channel_train_options.py",
    "../launch_4channel_training.py"
]

for file_path in files_created:
    full_path = os.path.abspath(file_path)
    exists = "✅" if os.path.exists(full_path) else "❌"
    print(f"   {exists} {file_path}")

print(f"\n📊 Dataset Summary:")
print(f"   Location: {output_dir}")
print(f"   Samples: {len(samples_list)}")
print(f"   MR channels: {mr_loaded.shape[-1] if len(mr_loaded.shape) == 4 else 1}")
print(f"   MR shape: {mr_loaded.shape}")
print(f"   CT shape: {ct_loaded.shape}")

print(f"\n🎯 Next Steps:")
print(f"   1. Navigate to the 3D directory: cd ..")
print(f"   2. Run training with: python launch_4channel_training.py")
print(f"   3. Monitor training: tensorboard --logdir checkpoints/")
print(f"   4. Use trained model for inference with test_npz.py")